In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import os
import re
import json
import tensorflow_datasets as tfds

In [2]:
def html_parser():
  list_html=[]
  for season in range(8):
    url = 'https://genius.com/albums/Game-of-thrones/Season-'+str(season+1)+'-scripts'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
      
    a = soup.findAll("a", {"class": "u-display_block"})
    list_links = [data['href'] for data in a]
    for key, link in enumerate(list_links):
      url = link
      response = requests.get(url)
      soup = BeautifulSoup(response.content, 'html.parser')
      a = soup.findAll("div", {"class": "lyrics"})
      list_html.append(a)
  return list_html


In [3]:
list_html=html_parser()

In [4]:
len(list_html)

75

In [ ]:
list_html.pop(40)
list_html.pop(51)

In [8]:
def scene_to_line_dict():
  
  r = re.compile("^\w+:|\w+\s\w+:")
  lines_dict={}
  n=0
  for e in range(73):
    
    if e==6:
      for i in list(list_html[e][0].findAll("p")[1:]):
        text= i.text.split('\n')
        n=n+1
        lines=[]
        for line in text:
          if re.match(r,line):
            lines.append(line)
          else:
            pass
          if lines!=[]:
            lines_dict[n]=lines


    elif e==7:
      for p,i in enumerate(list(list_html[e][0].findAll("p")[1:])):
        text= i.text.split('\n')
        n=n+1
        lines=[]
        for line in text:
          if (line.startswith('Back')):
            n=n+1
            lines=[]
            continue
          elif re.match(r,line):
            lines.append(line)
          else:
            pass
          if lines!=[]:
            lines_dict[n]=lines

    elif e==38:
      for p,i in enumerate(list(list_html[e][0].findAll("p")[1:])):
        text= i.text.split('\n')
        n=n+1
        lines=[]
        for line in text:
          if (line.startswith('[')):
            n=n+1
            lines=[]
            continue
          elif re.match(r,line):
            lines.append(line)
          else:
            pass
          if lines!=[]:
            lines_dict[n]=lines 
  
    else :
      for p,i in enumerate(list(list_html[e][0].findAll("p")[1:])):
        text=i.text.split('\n')
        n=n+1
        lines=[]
        for line in text:
          if (re.search('scene',line) or re.search('Scene',line) or re.match('EPISODE',line)
            or re.match('-',line) or re.match('CUT TO:',line) or re.match('INT:',line) 
            or re.match('EXT:',line) or re.match('EXT.',line) or re.match('INT.',line)) :

            n=n+1
            lines=[]
            continue
          elif re.match(r,line):
            lines.append(line)
          else:
            pass
          if lines!=[]:
            lines_dict[n]=lines
  with open('full_script.json','w') as f:
    json.dump(lines_dict,f)
  return lines_dict

In [9]:
line_dict=scene_to_line_dict()

IndexError: ignored

In [11]:
len_scene=[]
for k in list(line_dict.keys()):
  len_scene.append({'Scene':k,'Count':len(line_dict[k])})

pd.DataFrame(len_scene).sort_values(by='Count',ascending=False).head(15)

,Scene,Count
149,157,93
644,744,91
1341,1706,74
822,945,69
1276,1626,69
818,941,66
278,328,65
662,764,61
138,145,58
760,881,56


In [12]:
def preprocess_sentence(sentence):
  sentence = sentence.lower().strip()
  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = re.sub(r'[" "]+', " ", sentence)
  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)
  sentence = sentence.strip()
  # adding a start and an end token to the sentence
  return sentence
def load_dialouges():

  with open('full_script.json','r',encoding="utf-8") as f:
    data=json.load(f)

  input=[]
  output=[]

  for key in list(data.keys()):
    for i in range(len(data[key])-1):
      input.append(preprocess_sentence(data[key][i].split(':')[1]))
      output.append(preprocess_sentence(data[key][i+1].split(':')[1]))

  return input, output
  

In [25]:
questions, answers=load_dialouges()

In [1]:
print(questions[20])
print(answers[20])

NameError: ignored

In [12]:
tokenizer= tfds.features.text.SubwordTextEncoder.build_from_corpus(questions + answers,target_vocab_size=2**13)
start_token, end_token= [tokenizer.vocab_size], [tokenizer.vocab_size +1]

In [28]:
VOCAB_SIZE=tokenizer.vocab_size + 2

AttributeError: ignored

In [22]:
!pip install tokenizers

In [29]:
from transformers import RobertaTokenizer, TFRobertaForQuestionAnswering
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = TFRobertaForQuestionAnswering.from_pretrained('roberta-base')

Some weights of the model checkpoint at roberta-base were not used when initializing TFRobertaForQuestionAnswering: ['lm_head']
- This IS expected if you are initializing TFRobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFRobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFRobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
input_dict = tokenizer(questions, answers,padding=True,truncat return_tensors='tf')
start_scores, end_scores = model(input_dict)

In [18]:
start_token, end_token= [tokenizer.vocab_size], [tokenizer.vocab_size +1]

In [19]:
MAX_LENGTH = 40


# Tokenize, filter and pad sentences
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []
  
  for (sentence1, sentence2) in zip(inputs, outputs):
    # tokenize sentence
    sentence1 = start_token + tokenizer.encode(sentence1) + end_token
    sentence2 = start_token + tokenizer.encode(sentence2) + end_token
    # check tokenized sentence max length
    if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
      tokenized_inputs.append(sentence1)
      tokenized_outputs.append(sentence2)
  
  # pad tokenized sentences
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
  
  return tokenized_inputs, tokenized_outputs


questions, answers = tokenize_and_filter(questions, answers)

In [21]:
questions.shape

(17465, 40)

In [80]:
answers.shape

(17888, 40)

In [98]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

# decoder inputs use the previous target as input
# remove START_TOKEN from targets
dataset = tf.data.Dataset.from_tensor_slices((
    (
         questions,
        answers[:, :-1]
    ),
    (
         answers[:, 1:]
    ),
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [99]:
print(dataset)

<PrefetchDataset shapes: (((None, 40), (None, 39)), (None, 39)), types: ((tf.int32, tf.int32), tf.int32)>


In [1]:
pip install transformers

     |████████████████████████████████| 778kB 3.4MB/s 
     |████████████████████████████████| 890kB 15.3MB/s 
     |████████████████████████████████| 1.1MB 13.6MB/s 
     |████████████████████████████████| 3.0MB 35.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=e142304a27b0287707fe833d7fda0013aedd62ccad4fde0567109403a2de8369
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from transformers import TFBertForSequenceClassification
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier', 'dropout_37']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
pipeline=transformers.Pipeline(model,tokenizer,)

In [102]:
iterator=dataset.as_numpy_iterator()

In [70]:
train_stream=data.tf_inputs.data_streams(dataset,shuffle_buffer_size=BUFFER_SIZE)

AttributeError: ignored

In [54]:
from trax import data

In [88]:
data_pipeline=data.Serial(
    data.inputs.Log(),
    data.Shuffle(BUFFER_SIZE),
    data.Batch(BATCH_SIZE)
)

In [89]:
train_stream=data_pipeline(dataset)

In [48]:
print(dataset)

<BatchDataset shapes: ({inputs: (None, 40), dec_inputs: (None, 39)}, {outputs: (None, 39)}), types: ({inputs: tf.int32, dec_inputs: tf.int32}, {outputs: tf.int32})>


In [ ]:
!pip install -q -U trax
import trax

In [35]:
from trax.fastmath import numpy as fastnp
trax.fastmath.set_backend('tensorflow-numpy')
print(trax.fastmath.backend_name())

AttributeError: ignored

In [18]:
model=trax.models.transformer.Transformer(input_vocab_size=VOCAB_SIZE)

In [37]:
from trax.supervised import training

In [106]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  
  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

In [109]:
from trax import layers as tl
train_task = training.TrainTask(
    labeled_data=iterator,
    loss_layer=tl.CrossEntropyLoss(),
    optimizer=trax.optimizers.Adam(0.01),
    n_steps_per_checkpoint=500,
)

In [110]:
training_loop = training.Loop(model,
                              train_task)
                              #eval_tasks=[eval_task],
                              #output_dir=output_dir)

# Run 2000 steps (batches).
training_loop.run(2000)

LayerError: ignored

In [34]:
optimizer = tf.keras.optimizers.Adam(
    lr=0.001, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  # ensure labels have shape (batch_size, MAX_LENGTH - 1)
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

AttributeError: ignored

In [30]:
from trax import trax2keras
from trax import AsKeras

ImportError: ignored

In [26]:
model=trax2keras.AsKeras(model)

AttributeError: ignored

In [ ]:
pip install tf-models-official

In [24]:
import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from official.modeling import tf_utils
from official import nlp
from official.nlp import bert

# Load the required submodules
import official.nlp.optimization
import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization
import official.nlp.data.classifier_data_lib
import official.nlp.modeling.losses
import official.nlp.modeling.models
import official.nlp.modeling.networks

In [40]:
def encode_sentence(s, tokenizer):
   tokens = list(tokenizer.tokenize(s))
   tokens.append('[SEP]')
   return tokenizer.convert_tokens_to_ids(tokens)

def bert_encode(sentence1,sentence2, tokenizer):
  num_examples = len(sentence1)
  
  sentence1 = tf.ragged.constant([
      encode_sentence(s, tokenizer)
      for s in sentence1])
  sentence2 = tf.ragged.constant([
      encode_sentence(s, tokenizer)
       for s in sentence2])

  cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])]*sentence1.shape[0]
  input_word_ids = tf.concat([cls, sentence1, sentence2], axis=-1)

  input_mask = tf.ones_like(input_word_ids).to_tensor()

  type_cls = tf.zeros_like(cls)
  type_s1 = tf.zeros_like(sentence1)
  type_s2 = tf.ones_like(sentence2)
  input_type_ids = tf.concat(
      [type_cls, type_s1, type_s2], axis=-1).to_tensor()

  inputs = {
      'input_ids': input_word_ids.to_tensor(),
      'input_mask': input_mask,
      'input_type_ids': input_type_ids}

  return inputs

In [41]:
dataset= bert_encode(questions,answers,tokenizer)
for key, value in dataset.items():
  print(f'{key:15s} shape: {value.shape}')

input_ids       shape: (20759, 474)
input_mask      shape: (20759, 474)
input_type_ids  shape: (20759, 474)


In [ ]:
def evaluate(sentence):
  sentence = preprocess_sentence(sentence)

  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  output = tf.expand_dims(START_TOKEN, 0)

  for i in range(MAX_LENGTH):
    predictions = model(inputs=[sentence, output], training=False)

    # select the last word from the seq_len dimension
    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # return the result if the predicted_id is equal to the end token
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # concatenated the predicted_id to the output which is given to the decoder
    # as its input.
    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0)


def predict(sentence):
  prediction = evaluate(sentence)

  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('Input: {}'.format(sentence))
  print('Output: {}'.format(predicted_sentence))

  return predicted_sentence

In [ ]:
output = predict('Where have you been?')

Input: Where have you been?
Output: that s not a place to spend a lot of time wondering where s between my legs .


In [ ]:
output= predict('Dracarys')

Input: Dracarys
Output: he ll be able to feed himself when he wants .
